## Bayesian Model for Predicting Player Impact Based on College and Draft Position

In [47]:
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.static import players
import pandas as pd
import numpy as np

In [48]:
win_share_25 = pd.read_csv('Data/NBA Win Shares Leaders 24-25.csv')
#will specify starting year of season
win_share_25['Year'] = 2024
#rearrange columns for readibility
new_order = ['Rk','Year','Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'PER', 'TS%',
            '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
            'USG%', 'OWS', 'DWS', 'WS▼', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP',
            'Awards']
win_share_25 = win_share_25[new_order]
win_share_24 = pd.read_csv('Data/NBA Win Shares Leaders 23-24.csv')
win_share_24['Year'] = 2023
win_share_24 = win_share_24[new_order]
win_share_23 = pd.read_csv('Data/NBA Win Shares Leaders 22-23.csv')
win_share_23['Year'] = 2022
win_share_23 = win_share_23[new_order]
win_share_22 = pd.read_csv('Data/NBA Win Shares Leaders 21-22.csv')
win_share_22['Year'] = 2021
win_share_22 = win_share_22[new_order]
win_share_21 = pd.read_csv('Data/NBA Win Shares Leaders 20-21.csv')
win_share_21['Year'] = 2020
win_share_21 = win_share_21[new_order]

win_shares = pd.concat([win_share_25, win_share_24, win_share_23, win_share_22, win_share_21], ignore_index=True, sort=False)
win_shares.head()

,Rk,Year,Player,Age,Team,Pos,G,GS,MP,PER,...,USG%,OWS,DWS,WS▼,WS/48,OBPM,DBPM,BPM,VORP,Awards
0,1,2024,Shai Gilgeous-Alexander,26,OKC,PG,76,76,2598,30.7,...,34.8,11.9,4.8,16.7,0.309,8.9,2.6,11.5,8.9,"MVP-1,DPOY-10,CPOY-8,AS,NBA1"
1,2,2024,Nikola Jokić,29,DEN,C,70,70,2571,32.0,...,29.5,12.7,3.8,16.4,0.307,9.9,3.3,13.3,9.8,"MVP-2,CPOY-2,AS,NBA1"
2,3,2024,Ivica Zubac,27,LAC,C,80,80,2624,22.3,...,19.5,7.0,4.7,11.7,0.215,2.3,0.8,3.1,3.4,"DPOY-6,DEF2"
3,4,2024,Jarrett Allen,26,CLE,C,82,82,2296,22.1,...,15.9,7.6,4.0,11.6,0.243,3.0,1.2,4.2,3.6,NaN
4,5,2024,Giannis Antetokounmpo,30,MIL,PF,67,67,2289,30.5,...,35.2,7.8,3.7,11.5,0.241,6.9,2.5,9.5,6.6,"MVP-3,DPOY-8,AS,NBA1"


Next pull all college data and store as a dictionary

In [ ]:
active_players = np.unique(win_shares['Player'])

info_dict = {}

for player in active_players:
    try:
        player_info = commonplayerinfo.CommonPlayerInfo(player_id=players.find_players_by_full_name(player)[0]['id'])
        info = player_info.common_player_info.get_dict()
        info_df = pd.DataFrame(info['data'], columns=info['headers'])
        
        college = info_df.loc[0, 'SCHOOL']
        draft_pick = info_df.loc[0, 'DRAFT_NUMBER']
        
        info_dict[player] = {'College': college, 'Draft Pick': draft_pick}
    except:
        info_dict[player] = {'College': 'NA', 'Draft Pick': 'NA'}


In [ ]:
info_dict['Bam Adebayo']

{'College': 'NA', 'Draft Pick': 'NA'}

In [ ]:
players.find_players_by_full_name("Lebron James")

AttributeError: 'numpy.ndarray' object has no attribute 'find_players_by_full_name'

In [ ]:
player_info = commonplayerinfo.CommonPlayerInfo(player_id=players.find_players_by_full_name("Nikola Joic")[0]['id'])
info = player_info.common_player_info.get_dict()

AttributeError: 'numpy.ndarray' object has no attribute 'find_players_by_full_name'

In [33]:
df = pd.DataFrame(info['data'], columns=info['headers'])
df

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,PLAYER_SLUG,BIRTHDATE,SCHOOL,COUNTRY,...,PLAYERCODE,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,GREATEST_75_FLAG
0,203999,Nikola,Jokić,Nikola Jokić,"Jokić, Nikola",N. Jokić,nikola-jokić,1995-02-19T00:00:00,Mega Basket,Serbia,...,nikola_jokic,2015,2025,N,Y,Y,2014,2,41,N


In [29]:
df.columns

Index(['PERSON_ID', 'FIRST_NAME', 'LAST_NAME', 'DISPLAY_FIRST_LAST',
       'DISPLAY_LAST_COMMA_FIRST', 'DISPLAY_FI_LAST', 'PLAYER_SLUG',
       'BIRTHDATE', 'SCHOOL', 'COUNTRY', 'LAST_AFFILIATION', 'HEIGHT',
       'WEIGHT', 'SEASON_EXP', 'JERSEY', 'POSITION', 'ROSTERSTATUS',
       'GAMES_PLAYED_CURRENT_SEASON_FLAG', 'TEAM_ID', 'TEAM_NAME',
       'TEAM_ABBREVIATION', 'TEAM_CODE', 'TEAM_CITY', 'PLAYERCODE',
       'FROM_YEAR', 'TO_YEAR', 'DLEAGUE_FLAG', 'NBA_FLAG', 'GAMES_PLAYED_FLAG',
       'DRAFT_YEAR', 'DRAFT_ROUND', 'DRAFT_NUMBER', 'GREATEST_75_FLAG'],
      dtype='object')

In [3]:
players.find_players_by_first_name("Stephen")
player_info = commonplayerinfo.CommonPlayerInfo(player_id=201939)
p_dict = player_info.common_player_info.get_dict()

In [15]:
player_info.available_seasons.get_dict()
player_info.player_headline_stats.get_dict()

{'headers': ['PLAYER_ID',
  'PLAYER_NAME',
  'TimeFrame',
  'PTS',
  'AST',
  'REB',
  'PIE'],
 'data': [[201939, 'Stephen Curry', '2025-26', 28.6, 4.1, 3.5, 0]]}

In [14]:
p_dict['data']

[[201939,
  'Stephen',
  'Curry',
  'Stephen Curry',
  'Curry, Stephen',
  'S. Curry',
  'stephen-curry',
  '1988-03-14T00:00:00',
  'Davidson',
  'USA',
  'Davidson/USA',
  '6-2',
  '185',
  16,
  '30',
  'Guard',
  'Active',
  'Y',
  1610612744,
  'Warriors',
  'GSW',
  'warriors',
  'Golden State',
  'stephen_curry',
  2009,
  2025,
  'N',
  'Y',
  'Y',
  '2009',
  '1',
  '7',
  'Y']]

In [21]:
from nba_api.stats.endpoints import playerdashboardbyyearoveryear

dash = playerdashboardbyyearoveryear.PlayerDashboardByYearOverYear(player_id=201939)
yoy = dash.by_year_player_dashboard.get_dict()
df = pd.DataFrame(yoy['data'], columns=yoy['headers'])

In [31]:
df[['PLUS_MINUS','PLUS_MINUS_RANK','NBA_FANTASY_PTS','NBA_FANTASY_PTS_RANK']]

,PLUS_MINUS,PLUS_MINUS_RANK,NBA_FANTASY_PTS,NBA_FANTASY_PTS_RANK
0,19,14,580.3,16
1,299,8,2851.5,9
2,189,11,2956.5,8
3,268,9,2622.2,12
4,509,6,2756.0,11
5,244,10,3015.5,7
6,-45,16,189.7,17
7,689,4,3024.3,6
8,486,7,2235.2,14
9,1015,2,3446.6,3


Getting ids of all active players

In [9]:
players.get_active_players()

[{'id': 1630173,
  'full_name': 'Precious Achiuwa',
  'first_name': 'Precious',
  'last_name': 'Achiuwa',
  'is_active': True},
 {'id': 203500,
  'full_name': 'Steven Adams',
  'first_name': 'Steven',
  'last_name': 'Adams',
  'is_active': True},
 {'id': 1628389,
  'full_name': 'Bam Adebayo',
  'first_name': 'Bam',
  'last_name': 'Adebayo',
  'is_active': True},
 {'id': 1630534,
  'full_name': 'Ochai Agbaji',
  'first_name': 'Ochai',
  'last_name': 'Agbaji',
  'is_active': True},
 {'id': 1630583,
  'full_name': 'Santi Aldama',
  'first_name': 'Santi',
  'last_name': 'Aldama',
  'is_active': True},
 {'id': 1641725,
  'full_name': 'Trey Alexander',
  'first_name': 'Trey',
  'last_name': 'Alexander',
  'is_active': True},
 {'id': 1629638,
  'full_name': 'Nickeil Alexander-Walker',
  'first_name': 'Nickeil',
  'last_name': 'Alexander-Walker',
  'is_active': True},
 {'id': 1628960,
  'full_name': 'Grayson Allen',
  'first_name': 'Grayson',
  'last_name': 'Allen',
  'is_active': True},
 {'id